In [5]:
import base64
import requests
import json
from utils import *

In [10]:
# Function to encode the image to base64
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        encoded_string = base64.b64encode(image_file.read()).decode()
    return encoded_string

def get_metadata_gpt(image_name):

    '''
    PROMPT TO GPT
    ------------------

    Based on the plot:

    1. What is the title of the x-axis?
    2. What is the range of the x-axis?
    3. What is the title of the y-axis?
    4. What is the range of the y-axis?
    5. What are the different types?

    Can you provide the answers to the aforementioned questions in the following format:

    answers = {
    "x-axis": {
    "title": [INSERT TITLE HERE IN STRING FORMAT],
    "range":  [INSERT RANGE HERE IN NUMERICAL FORMAT],
    },
    "y-axis": {
    "title": [INSERT TITLE HERE IN STRING FORMAT],
    "range":  [INSERT RANGE HERE IN NUMERICAL FORMAT],
    },
    "types": [INSERT THE TYPES IN LIST OF LISTS OF LENGTH = 2 WHERE EACH LIST LOOKS LIKE [TYPE NAME, MARKER COLOR in "black", "white", "red","green","blue", "yellow", "magenta", "cyan", "grey"]]
    }

    '''

    answers = None
    
    # Encode the image
    encoded_image = encode_image(image_name)

    headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {API_KEY}"
    }

    payload = {
    "model": "gpt-4-vision-preview",
    "messages": [
        {
        "role": "user",
        "content": [
            {
            "type": "text",
            "text": "Based on the plot: 1. Describe the title of the x-axis? 2. Describe the range of the x-axis? 3. Describe the title of the y-axis 4. Describe the range of the y-axis? 5. Describe the different types? Can you provide the answers to the aforementioned questions in the following format: answers = { \"x-axis\": { \"title\": [INSERT TITLE HERE IN STRING FORMAT], \"range\":  [INSERT RANGE HERE IN NUMERICAL FORMAT],}, \"y-axis\": { \"title\": [INSERT TITLE HERE IN STRING FORMAT], \"range\":  [INSERT RANGE HERE IN NUMERICAL FORMAT], }, \"types\": [INSERT THE TYPES IN LIST OF TUPLES OF LENGTH = 2 WHERE EACH TUPLE LOOKS LIKE (TYPE NAME, MARKER COLOR in \"black\", \"white\", \"red\",\"green\",\"blue\", \"yellow\", \"magenta\", \"cyan\", \"grey\")]}"
            },
            {
            "type": "image_url",
            "image_url": {
                "url": f"data:image/jpeg;base64,{encoded_image}"
            }
            }
        ]
        }
    ],
    "max_tokens": 300
    }

    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

    # Check if the response is successful
    if response.status_code == 200:
        print("here")
        # Parse the response as JSON
        answers = response.json()
    else:
        # Handle errors (non-200 responses)
        print(f"Error: {response.status_code} - {response.text}")

    return answers

In [7]:
image_number = 5

In [8]:
image_name = '../plot_images/'+str(image_number)+'.png'
answers = get_metadata_gpt(image_name)

here


In [9]:
answers

{'id': 'chatcmpl-8PatahZKY0oMGFivSFbrd7WmYunbP',
 'object': 'chat.completion',
 'created': 1701110718,
 'model': 'gpt-4-1106-vision-preview',
 'usage': {'prompt_tokens': 457,
  'completion_tokens': 142,
  'total_tokens': 599},
 'choices': [{'message': {'role': 'assistant',
    'content': 'The image shows a plot with labels for the axes and a legend describing two types of data series. Here are the details as per your request:\n\nanswers = {\n  "x-axis": {\n    "title": "Cycle number",\n    "range": [0, 100]\n  },\n  "y-axis": {\n    "title": "Coulombic Efficiency (%)",\n    "range": [0, 120]\n  },\n  "types": [\n    ("1.2 mol L^-1 LiPF_6 in FEC/FEMC/TFEB", "magenta"),\n    ("1 mol L^-1 LiPF_6 in EC/DMC", "blue")\n  ]\n}'},
   'finish_details': {'type': 'stop', 'stop': '<|fim_suffix|>'},
   'index': 0}]}